In [12]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person, SnapshotObjectType, OperationStatusType
import csv
import errno
import json
KEY = 'your key here'

# Set the FACE_ENDPOINT environment variable with the endpoint from your Face service in Azure.
# This endpoint will be used in all examples in this quickstart.
ENDPOINT = 'your endpoint here'
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))


In [13]:
PERSON_GROUP_ID = 'lab-group'

# Used for the Snapshot and Delete Person Group examples.
TARGET_PERSON_GROUP_ID = str(uuid.uuid4()) # assign a random ID (or name it anything)

# Create empty Person Group. Person Group ID must be lower case, alphanumeric, and/or with '-', '_'.
print('Person group:', PERSON_GROUP_ID)
face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

Person group: lab-group


APIErrorException: (PersonGroupExists) Person group 'lab-group' already exists.

In [35]:
#face_client.person_group.create(person_group_id=PERSON_GROUP_ID, name=PERSON_GROUP_ID)

# Define person friend
josh = face_client.person_group_person.create(PERSON_GROUP_ID, "josh")
# Define man friend
avery = face_client.person_group_person.create(PERSON_GROUP_ID, "avery")

josh_images = [file for file in glob.glob('*.jpg') if file.startswith("josh")]
for image in josh_images:
    a = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, josh.person_id, a)
    
avery_images = [file for file in glob.glob('*.jpg') if file.startswith("avery")]
for image in avery_images:
    a = open(image, 'r+b')
    face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, avery.person_id, a)

# Add to a child person
#for image in child_images:
    #ch = open(image, 'r+b')
    #face_client.person_group_person.add_face_from_stream(PERSON_GROUP_ID, child.person_id, ch)

    '''
Train PersonGroup
'''
print()
print('Training the person group...')
# Train the person group
face_client.person_group.train(PERSON_GROUP_ID)
print(josh.person_id)
print(avery.person_id)
while (True):
    training_status = face_client.person_group.get_training_status(PERSON_GROUP_ID)
    print("Training status: {}.".format(training_status.status))
    print()
    if (training_status.status is TrainingStatusType.succeeded):
        break
    elif (training_status.status is TrainingStatusType.failed):
        sys.exit('Training the person group has failed.')
        
PERSON_GROUP_ID = 'lab-group'
# Group image for testing against
group_photo = 'group_photo.jpg'
IMAGES_FOLDER = os.path.join(os.path.dirname(os.path.realpath(group_photo)))
# Get test image
test_image_array = glob.glob(os.path.join(IMAGES_FOLDER, group_photo))
image = open(test_image_array[0], 'r+b')

# Detect faces
person_ids = [avery.person_id,josh.person_id]
person_ids_dict = {josh.person_id:"joshua",avery.person_id:"avery"}
faces = face_client.face.detect_with_stream(image)
count = 0

for face in faces:
    # Identify faces
            for person in person_ids:
                results = face_client.face.verify_face_to_person(face.face_id, person, person_group_id=PERSON_GROUP_ID)
                if results.is_identical == True:
                    print(person_ids_dict[person])
                    val = 1
                    break
                else:
                    val = 0
            if val == 0:
                print("unknown")  




Training the person group...
17f82e77-389b-4a91-a812-670f1f807875
32c18574-c9a4-4330-9525-878cb9173bd7
Training status: running.

Training status: running.

Training status: running.

Training status: running.

Training status: running.

Training status: succeeded.

avery
joshua
